# Crypto Currency Data From CoinGecko API

## Imported Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
from bs4 import BeautifulSoup

## Log Progress Fuction

In [ ]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format='%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./log_progress","a") as f:
        f.write(f"{timestamp}:{message}" + '\n')

## Extract Fuction

This function retrieves cryptocurrency data from the CoinGecko API and saves it to a pandas DataFrame. The returned DataFrame is formatted for further analysis or processing.

Specifically, the function extracts information about the top 100 cryptocurrencies, including their ID, name, current price in USD, and market capitalization in USD.

In [16]:
def extract(url):    # sourcery skip: raise-specific-error
    ''' This function aims to extract the required
    information from the CoinGecko and save it to a data frame. The
    function returns the data frame for further processing. '''
    params = {  
               'vs_currency': 'USD'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data from API. Status code: {response.status_code}")
    data = response.json()
    df = pd.DataFrame(data)
    columns = ['id', 'name', 'current_price', 'market_cap']
    return df[columns]

extract("https://api.coingecko.com/api/v3/coins/markets?x_cg_demo_api_key=CG-MbEY8jPE4gh6VQGrJrCCF5st")

,id,name,current_price,market_cap
0,bitcoin,Bitcoin,99366.000000,1965862285317
1,ethereum,Ethereum,3079.410000,370167865786
2,ripple,XRP,2.920000,168125945799
3,tether,Tether,0.999567,139279136076
4,solana,Solana,227.610000,110441338396
...,...,...,...,...
95,jasmycoin,JasmyCoin,0.025009,1207888327
96,msol,Marinade Staked SOL,287.510000,1205086496
97,solana-swap,Solana Swap,1.210000,1204470317
98,dexe,DeXe,20.850000,1189192010


## Transform the current price and market cap to EUR and GBP

This function accesses a CSV file containing exchange rate information and adds three new columns to the DataFrame. These columns represent the transformed values of the 'Market Cap' and 'Current Price' columns into their respective currencies.

The function utilizes the `exchange_rate.csv` file to obtain the current exchange rates for USD to EUR and USD to GBP


In [27]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    # Get the exchange rate from the csv file
    exchangerate_df = pd.read_csv(csv_path)
    # Transform the exchange rate in the data frame to a dictionary, in order manipulate it.
    exchange_rate = exchangerate_df.set_index('Currency').to_dict()['Rate']

    # Added new columns
    df['current_price_GBP'] = [np.round(x*exchange_rate['GBP'],2) for x in df['current_price']]
    df['current_price_EUR'] = [np.round(x*exchange_rate['EUR'],2) for x in df['current_price']]
    df['market_cap_GBP'] = [np.round(x*exchange_rate['GBP'],2) for x in df['market_cap']]
    df['market_cap_EUR'] = [np.round(x*exchange_rate['EUR'],2) for x in df['market_cap']]
    return df

In [28]:
df = extract("https://api.coingecko.com/api/v3/coins/markets?x_cg_demo_api_key=CG-MbEY8jPE4gh6VQGrJrCCF5st")
exchange_rate_csv = './exchange_rate.csv'
df2 = transform(df, exchange_rate_csv)
df2

{'EUR': 0.95, 'GBP': 0.8}


,id,name,current_price,market_cap,current_price_GBP,current_price_EUR,market_cap_GBP,market_cap_EUR
0,bitcoin,Bitcoin,99443.000000,1965313117410,79554.40,94470.85,1.572250e+12,1.867047e+12
1,ethereum,Ethereum,3062.750000,368520498727,2450.20,2909.61,2.948164e+11,3.500945e+11
2,ripple,XRP,2.900000,166734447163,2.32,2.76,1.333876e+11,1.583977e+11
3,tether,Tether,0.999517,139267793872,0.80,0.95,1.114142e+11,1.323044e+11
4,solana,Solana,225.070000,109256404464,180.06,213.82,8.740512e+10,1.037936e+11
...,...,...,...,...,...,...,...,...
95,solana-swap,Solana Swap,1.200000,1203608514,0.96,1.14,9.628868e+08,1.143428e+09
96,msol,Marinade Staked SOL,284.040000,1195459824,227.23,269.84,9.563679e+08,1.135687e+09
97,jasmycoin,JasmyCoin,0.024702,1193361405,0.02,0.02,9.546891e+08,1.133693e+09
98,dexe,DeXe,20.770000,1183867379,16.62,19.73,9.470939e+08,1.124674e+09
